In [3]:
import pandas as pd

In [4]:
df1 = pd.read_csv("female_gothic_data/male_list_all_retrieved_texts.csv")
df2 = pd.read_csv("female_gothic_data/female_list_all_retrieved_texts.csv")
list(df1)

['\ufeffid',
 'source',
 'source_date_accessed',
 'source_id',
 'short_title',
 'author',
 'year',
 'genre',
 'gender',
 'translator',
 'title',
 'publication',
 'format',
 'catalogue_numbers',
 'notes',
 'further_editions',
 'riddell',
 'gutenberg',
 'underwood']

In [5]:
#for each row, check if underwood
df1_underwood = df1.loc[df1['underwood'] == 'x']
df2_underwood = df2.loc[df1['underwood'] == 'x']
#check if only riddell
df1_riddell = df1.loc[df1['underwood'] != 'x']
df1_riddell = df1_riddell.loc[df1_riddell['riddell'] == 'x']
df2_riddell = df2.loc[df2['underwood'] != 'x']
df2_riddell = df2_riddell.loc[df2_riddell['riddell'] == 'x']
#if yes, get txt by id column from female_gothic_txt and male_gothic_txt

#add a column for the riddell id so there's no chance of overlap with other id sets 
df1_riddell['rid'] = ["r_"+str(i) for i in list(df1_riddell['\ufeffid'])]
df2_riddell['rid'] = ["r_"+str(i) for i in list(df2_riddell['\ufeffid'])]

#check if matched in gutenberg only
df1_gutenberg = df1.loc[df1['underwood'] != 'x']
df1_gutenberg = df1_gutenberg.loc[df1_gutenberg['riddell'] != 'x']

df2_gutenberg = df2.loc[df2['underwood'] != 'x']
df2_gutenberg = df2_gutenberg.loc[df2_gutenberg['riddell'] != 'x']

#if yes, title match in gutenberg id male_gothic_matches_filtered, female_gothic_matches_filtered
gut_match_1 = pd.read_csv("female_gothic_data/male_gothic_matches_filtered.csv")
gut_match_2 = pd.read_csv("female_gothic_data/female_gothic_matches_filtered.csv")
#read more than one match as multivolume

In [6]:
#convert each df to a list of dicts in id order, join with existing lists, genre_
"""
_ids, dates, genres, authors, titles, big_genres, lavin_genres, processed_genres 
"""
#big_genres value = important
#append columns to pData attributes
#append dictionaries to feature_dicts_full
#remove all underwood biggenre = gothic

'\n_ids, dates, genres, authors, titles, big_genres, lavin_genres, processed_genres \n'

In [7]:
#get txt files for all gutenberg_match files
#0 = gut_id
#1 = title len 25
# url is structured like this http://www.gutenberg.org/cache/epub/[id]/pg[id].txt
#if yes, title match in gutenberg id male_gothic_matches_filtered, female_gothic_matches_filtered
gut_match_1 = pd.read_csv("female_gothic_data/male_gothic_matches_filtered.csv")
gut_match_2 = pd.read_csv("female_gothic_data/female_gothic_matches_filtered.csv")
import requests
def get_gut(idlist, folder):
    base_url = "http://www.gutenberg.org/cache/epub/"
    #[id]/pg[id].txt
    for gid in idlist:
        gids = str(gid)
        gut_url = base_url + gids +"/pg" + gids + ".txt"
        gut = requests.get(gut_url)
        gtxt = gut.text
        filename = folder + "/g_" + gids + ".txt"
        with open(filename, "a") as f:
            f.write(gtxt)
        f.close()
#run function on id lists, folders
get_gut(list(gut_match_1['0']), "female_gothic_data/male_gothic_txt")
get_gut(list(gut_match_2['0']), "female_gothic_data/female_gothic_txt")
#done!

In [10]:
from random import shuffle
from application.selective_features import make_feature_list, dictionaries_of_features, make_genres_big_and_lavin
from application.pickles import pickledData
import pandas as pd
from scipy.stats.stats import spearmanr
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import recall_score

# all this has to do is split a list of ids into three groups
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def divide_by_three(list_of_ids):
    destinations = []
    while len(list_of_ids) > 2 or len(list_of_pairs) % 3 == 0:
        #grab a random id from big_genre and remove
        for m, l in enumerate(destinations):

            candidates = list(list_of_pairs.keys())
            shuffle(candidates)
            key = candidates[0]
            #print(key)
            candidate = list_of_pairs[key]
            #remove from dictionary
            del list_of_pairs[key]
            #make sure the author is not in the dict
            if candidate not in l:
                l[key] = candidate

            else:
                next_one = destinations[m+1]
                #next list same id if author in dict
                if candidate not in next_one:
                    next_one[key] = candidate
                else:
                    next_one = destinations[m+2]
                    if candidate not in next_one:
                        next_one[key] = candidate
    destinations_trimmed =[]
    for i,j in enumerate(destinations):
        aTuple = list(j.items())
        shuffle(aTuple)
        destinations_trimmed.append(dict(aTuple[:83]))
    return destinations_trimmed

def divide_list(mylist, number_of_splits):
    def chunks(l, n):
        """Yield successive n-sized chunks from l."""
        for i in range(0, len(l), n):
            yield l[i:i + n]

    l = (len(mylist) % number_of_splits)
    a = len(mylist) - l
    b = int(a/number_of_splits)
    newlists = list(chunks(mylist, b))
    newlists = [i for i in newlists if len(i) == b]
    return newlists

#this will be moved to application folder
def partition_test_train(genre, df, number_of_partitions):
    # supply a genre
    df_genre = df.loc[df['big_genres'] == genre]
    df_non_genre = df.loc[df['big_genres'] != genre ]
    df_non_genre = df_non_genre.loc[df_non_genre['big_genres'] != "multi" ]
    # create a dataset of random ids 50% in genre, 50% out of genre
    genre_ids = list(df_genre['ids'])
    # split into three separate sets of ids
    genre_split = divide_list(genre_ids, number_of_partitions)

    non_genre_ids = list(df_non_genre['ids'])
    #shuffle non_genre before splitting
    shuffle(non_genre_ids)
    non_genre_ids = non_genre_ids[0:len(genre_ids)]
    # split into three separate sets of ids
    non_genre_split = divide_list(non_genre_ids, number_of_partitions)

    rejoined = []
    #rejoin genre and nongenre

    for i, j in enumerate(non_genre_split):
        joined = j + genre_split[i]
        rejoined.append(joined)
    #sort by id and convert rejoined back to full df
    for r in rejoined:
        r.sort()
    rejoined_dfs = []
    for i in rejoined:
        new_df = df.loc[df['ids'].isin(i)]
        rejoined_dfs.append(new_df)
    return rejoined_dfs, rejoined

pData = pickledData()

# import feature dictionaries
feature_dicts_full = pData.feature_dicts

photo_terms = make_feature_list("lavin_lexicon/photo_terms.csv", "score", 8)

#perform feature selection on dictionaries of term frequencies, using top 100 photo_terms
feature_dicts = dictionaries_of_features(feature_dicts_full, photo_terms)

#define a dictionary with ids, genres, dates, authors, etc. Pass to each function
metadata = {"ids": pData._ids, "dates": pData.dates, "genres":pData.genres, "authors":pData.authors,
                "titles":pData.titles,"big_genres":pData.big_genres, "lavin_genres":pData.lavin_genres}

#convert all metadata to dataframe
data = [metadata[i] for i in metadata.keys()]
df = pd.DataFrame(data)
df = df.transpose()
df.index.name = 'position'
df.columns=metadata.keys()

df1 = pd.read_csv("female_gothic_data/male_list_all_retrieved_texts.csv")
df2 = pd.read_csv("female_gothic_data/female_list_all_retrieved_texts.csv")

#for each row, get underwood matches
df1_underwood = df1.loc[df1['underwood'] == 'x']
df2_underwood = df2.loc[df2['underwood'] == 'x']

#check if only riddell or gutenberg (if both, use riddell)
df1_riddell = df1.loc[df1['underwood'] != 'x']
df1_riddell = df1_riddell.loc[df1_riddell['riddell'] == 'x']

df2_riddell = df2.loc[df2['underwood'] != 'x']
df2_riddell = df2_riddell.loc[df2_riddell['riddell'] == 'x']

#add a column for the riddell id so there's no chance of overlap with other id sets
df1_riddell['rid'] = ["r_"+str(i) for i in list(df1_riddell['\ufeffid'])]
df2_riddell['rid'] = ["r_"+str(i) for i in list(df2_riddell['\ufeffid'])]

#check if matched in gutenberg only
df1_gutenberg = df1.loc[df1['underwood'] != 'x']
df1_gutenberg = df1_gutenberg.loc[df1_gutenberg['riddell'] != 'x']

df2_gutenberg = df2.loc[df2['underwood'] != 'x']
df2_gutenberg = df2_gutenberg.loc[df2_gutenberg['riddell'] != 'x']

#add a column for the gutenberg id so there's no chance of overlap with other id sets
df1_gutenberg['gid'] = ["g_"+str(i) for i in list(df1_gutenberg['\ufeffid'])]

df2_gutenberg['gid'] = ["g_"+str(i) for i in list(df2_gutenberg['\ufeffid'])]

#convert each df to a list of dicts in id order, join with existing lists
"""
_ids, dates, genres, authors, titles, big_genres, lavin_genres, processed_genres
"""
#big_genres value = important
#append columns to pData attributes
#append dictionaries to feature_dicts_full

def for_append_metadata(df, id_column, gender, metadata):
    _ids = list(df[id_column])
    dates = list(df["year"])
    genres = ["gothic" for i in _ids]
    authors = list(df["author"])
    titles = list(df["title"])
    big_genres = [gender + "_gothic" for i in _ids]
    lavin_genres = ["gothic" for i in _ids]
    columns = [titles, lavin_genres, big_genres, _ids, authors, genres, dates]
    df = pd.DataFrame(columns)
    df = df.transpose()
    df.columns=metadata.keys()
    df.index.name = 'position'
    
    return df

df_ = for_append_metadata(df1_riddell, 'rid', "male", metadata)
df = df.append(df_)
df_ = for_append_metadata(df1_gutenberg, 'gid', "male", metadata)
df = df.append(df_)
df_ = for_append_metadata(df1_underwood, '\ufeffid', "male", metadata)
df = df.append(df_)
df_ = for_append_metadata(df1_gutenberg, 'gid', "female", metadata)
df = df.append(df_)
df_ = for_append_metadata(df2_riddell, 'rid', "female", metadata)
df = df.append(df_)
df_ = for_append_metadata(df2_underwood, '\ufeffid', "female", metadata)
df = df.append(df_)

Loaded pickle data successfully.


In [11]:
df = df.loc[df["big_genres"] != "gothic"]
df

,titles,ids,authors,dates,genres,lavin_genres,big_genres
position,,,,,,,
0,The+savages+of+Europe,1,Lesuire%2C+Robert+Martin%2C,1760,teamred | random,no_lavin_tag,non_genre
1,Eloisa%3A+or+A+series+of+or,2,Rousseau%2C+Jean-Jacques%2C,1761,teamred | random,no_lavin_tag,non_genre
2,The+memoirs+of+Miss+D%27Arv,3,Anonymous4,1764,teamblack | random,no_lavin_tag,non_genre
4,Memoirs+of+the+year+two+t,5,Mercier%2C+Louis-Sbastien%2C,1771,teamblack | anatscifi,no_lavin_tag,scifi
5,Letters+from+Henrietta+to,6,Henrietta.,1777,teamblack | random,no_lavin_tag,non_genre
7,Letters+of+Madame+Du+Montier,8,Leprince+de+Beaumont%2C,1780,teamblack | random,no_lavin_tag,non_genre
8,The+new+Robinson+Crusoe%3B,9,Campe%2C+Joachim+Heinrich%2C,1780,teamblack | random,no_lavin_tag,non_genre
9,Tales+of+the+castle%3B,10,Genlis%2C+Stphanie+Flicit,1785,teamred | random,no_lavin_tag,non_genre
11,Anna%3B+or%2C+Memoirs+of+a+Welch+Heiress,12,Bennett%2C+Anna+Maria%2C,1786,teamred | random,no_lavin_tag,non_genre


In [12]:
metadata.keys()

dict_keys(['titles', 'ids', 'authors', 'dates', 'genres', 'lavin_genres', 'big_genres'])

In [13]:
culled_dicts = []
df_originals = df.loc[df['big_genres'] != 'female_gothic']
df_originals = df_originals.loc[df_originals['big_genres'] != 'male_gothic']
df_culled = list(df_originals['ids'])
for h,i in enumerate(pData._ids):
    if i in df_culled:
        culled_dicts.append(feature_dicts_full[h])

In [23]:
df['ids']

position
0          1
1          2
2          3
4          5
5          6
7          8
8          9
9         10
11        12
13        14
16        17
18        19
20        21
24        25
25        26
28        29
29        30
32        33
36        37
37        38
38        39
39        40
40        41
41        42
42        43
45        46
46        47
49        50
51        52
55        56
       ...  
35    gothic
36    gothic
37    gothic
38    gothic
39    gothic
40    gothic
41    gothic
42    gothic
43    gothic
44    gothic
45    gothic
46    gothic
47    gothic
48    gothic
0     gothic
1     gothic
2     gothic
3     gothic
4     gothic
5     gothic
6     gothic
7     gothic
8     gothic
9     gothic
10    gothic
0     gothic
1     gothic
2     gothic
3     gothic
4     gothic
Name: ids, dtype: object

In [16]:
g_ids = list(gut_match_1['0'])
g_titles = list(gut_match_1['1'])

In [18]:
#merge multivolume together, title:list_of_ids even if just one id
merged = {}
for r,t in enumerate(g_titles):
    try:
        merged[t].append(t)
    except:
        merged[t] = g_ids[r]